In [1]:
from py_files import preprocessing as pp
from py_files import Augmentation as ag
from py_files import pic_class as pc
from py_files import TrainTestSplit as ts
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib agg
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import zipfile
import os

Using TensorFlow backend.


In [10]:
source = os.getcwd()
source

'/mnt/c/Users/nozom/desktop/Spicy-Raman/model_train_and_validate'

## Data Importing

In [11]:
train_dir = os.path.join(source, 'trainingdata_Pictures')
validation_dir = os.path.join(source, 'validationdata_Pictures')

## Data Information

In [12]:
train_fluorescent_dir = os.path.join(train_dir, 'fluorescent_Pictures')
train_nylon_dir = os.path.join(train_dir, 'nylon_Pictures')
train_polyamide_dir = os.path.join(train_dir, 'polyamide_Pictures')
train_polyethylene_dir = os.path.join(train_dir, 'polyethylene_Pictures')
train_others_dir = os.path.join(train_dir, 'others_Pictures')

validation_fluorescent_dir = os.path.join(validation_dir, 'fluorescent_Pictures')
validation_nylon_dir = os.path.join(validation_dir, 'nylon_Pictures')
validation_polyamide_dir = os.path.join(validation_dir, 'polyamide_Pictures')
validation_polyethylene_dir = os.path.join(validation_dir, 'polyethylene_Pictures')
validation_others_dir = os.path.join(validation_dir, 'others_Pictures')

num_fluorescent_tr = len(os.listdir(train_fluorescent_dir))
num_nylon_tr = len(os.listdir(train_nylon_dir))
num_polyamide_tr = len(os.listdir(train_polyamide_dir))
num_polyethylene_tr = len(os.listdir(train_polyethylene_dir))
num_others_tr = len(os.listdir(train_others_dir))

num_fluorescent_val = len(os.listdir(validation_fluorescent_dir))
num_nylon_val = len(os.listdir(validation_nylon_dir))
num_polyamide_val = len(os.listdir(validation_polyamide_dir))
num_polyethylene_val = len(os.listdir(validation_polyethylene_dir))
num_others_val = len(os.listdir(validation_others_dir))


#num_cats_val = len(os.listdir(validation_cats_dir))
#num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_fluorescent_tr + num_polyamide_tr + num_nylon_tr + num_polyethylene_tr + num_others_tr
total_val = num_fluorescent_val + num_polyamide_val + num_nylon_val + num_polyethylene_val + num_others_val

#validation_cats_dir = os.path.join(validation_dir, 'cats')
#validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print("Total training images:", total_train)
print("Total validation images:", total_val)

Total training images: 463
Total validation images: 122


## Data Training

In [17]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [18]:
train_image_generator = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [19]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 463 images belonging to 5 classes.


In [20]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 122 images belonging to 5 classes.


In [21]:
sample_training_images, _ = next(train_data_gen)

In [22]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [23]:
plotImages(sample_training_images[:5])

In [24]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [25]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 3 steps, validate for 1 steps
Epoch 1/15
3/3 [==============================] - 11s 4s/step - loss: -17.3835 - accuracy: 0.1552 - val_loss: -78.3181 - val_accuracy: 0.1393
Epoch 2/15
3/3 [==============================] - 8s 3s/step - loss: -145.7600 - accuracy: 0.1522 - val_loss: -385.6600 - val_accuracy: 0.1393
Epoch 3/15
3/3 [==============================] - 9s 3s/step - loss: -625.3279 - accuracy: 0.1667 - val_loss: -1278.0463 - val_accuracy: 0.1393
Epoch 4/15
3/3 [==============================] - 9s 3s/step - loss: -1755.7723 - accuracy: 0.1672 - val_loss: -3471.0845 - val_accuracy: 0.1393
Epoch 5/15
3/3 [==============================] - 9s 3s/step - loss: -4519.2677 - accuracy: 0.1672 - val_loss: -8275.7607 - val_accuracy: 0.1393
Epoch 6/15
3/3 [==============================] - 9s 3s/step - loss: -11188.7791 - accuracy: 0.1522 - val_loss: -17

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("Spicy_Raman_Saved_Model")
zipfile.ZipFile("Spicy_Raman_Saved_Model.zip", mode='w').write("Spicy_Raman_Saved_Model.h5", compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
class_names = ['Polyethylene', 'Fluorescent', 'Polyamide', 'Nylon', 'Unknown']
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (28,28)),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.Dense(10, activation = "softmax")
    ])

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])